## Samsum dataset
- Easy access
- Decent quality
- Some commercial limitations noted: https://huggingface.co/datasets/samsum

In [ ]:
!pip install py7zr
!pip list | grep 'py7zr'

In [ ]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("samsum")
samsum = pd.DataFrame(dataset["test"])
samsum.iloc[0].dialogue.split("\n")


## GPT generative conversation
- Using microsoft/DialoGPT-medium and large
- Pytorch weight loading issues noted with blenderbot large/small variants: https://huggingface.co/transformers/model_doc/blenderbot.html

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")


In [ ]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(
        input(">> User:") + tokenizer.eos_token, return_tensors="pt"
    )

    # append the new user input tokens to the chat history
    bot_input_ids = (
        torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
        if step > 0
        else new_user_input_ids
    )

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id
    )

    # pretty print last ouput tokens from bot
    print(
        "DialoGPT: {}".format(
            tokenizer.decode(
                chat_history_ids[:, bot_input_ids.shape[-1] :][0],
                skip_special_tokens=True,
            )
        )
    )


from transformers import BlenderbotSmallTokenizer, BlenderbotSmallForConditionalGeneration

mname = 'facebook/blenderbot_small-90M'
model = BlenderbotSmallForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotSmallTokenizer.from_pretrained(mname)
UTTERANCE = "My friends are cool but they eat too many carbs."
print("Human: ", UTTERANCE)
inputs = tokenizer([UTTERANCE], return_tensors='pt')

reply_ids = model.generate(**inputs)
print("Bot: ", tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0])
# what kind of carbs do they eat? i don't know much about carbs.

## ParlAI
- Options for notebooks/source as well as CLI scripts configured across a variety of tasks
- Able to create CLI chat interface, with human in the loop (chit-chat task)
- Able to loop model back onto itself (model-model), useful for programmatically generating large conversational datasets?
- Grammar/sentence formation is an issue, though this is not an issue in terms of down-stream TTS

In [ ]:
!pip install parlai

In [ ]:
# interactive prompts
parlai interactive --model-file zoo:blender/blender_90M/model --task convai2

In [ ]:
# self chat
parlai self_chat --model-file zoo:blender/blender_90M/model --task convai2 --inference topk --num-self-chats 10 --display-examples True --datatype valid

In [ ]:
# seed with below conversation_starters text file
/home/samhardyhey/.local/bin/parlai self_chat --model-file zoo:blender/blender_90M/model --task convai2 --inference topk --num-self-chats 2 --selfchat-max-turns 6 --display-examples True --datatype valid --seed-messages-from-file /home/samhardyhey/conversation_starters.txt

In [ ]:
%%writefile /home/samhardyhey/conversation_starters.txt
Ponzi scheme insurer
Time wasters and SUPER expensive
Bramdon was an excellent customer service provider rather than an agent.